# Final EDA and Modeling

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from created_functions import remove_dups,reason_cleanup,to_binary

pd.set_option('display.max_columns', 100)

In [26]:
#loading dataframes
df1=pd.read_excel("EDA_&_Modeling/Telco_customer_churn.xlsx")
df2=pd.read_excel("EDA_&_Modeling/Telco_customer_churn_status.xlsx")
df3=pd.read_excel("EDA_&_Modeling/Telco_customer_churn_services.xlsx")
df4=pd.read_excel("EDA_&_Modeling/Telco_customer_churn_demographics.xlsx")

In [27]:
#Mergin dataframes into one
df1_2=pd.merge(df1, df2, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)
df3_4=pd.merge(df3, df4, left_on='Customer ID', right_on='Customer ID', how='left')
customer_df=pd.merge(df1_2, df3_4, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)

In [28]:
#Cleaning up duplicates 
customer_df=customer_df.T.drop_duplicates().T
customer_df.columns=[column.lower().strip("_x").strip("_y").replace(" ","_")for column in customer_df.columns] 

In [29]:
#Removing duplicates that were not caught by the previous function.
remove_dup=["internet_service","online_securit","online_backup","multiple_lines","streaming_tv","payment_method","total_charges","contract","streaming_movies","churn_reason"]
remove_dups(remove_dup,customer_df)

In [30]:
#Dropping Redundant columns
customer_df.drop(["lat_long","customerid","churn_label","count","countr","state","quarter",],axis=1,inplace=True)

In [31]:
#Cleaning missing values from "total_charge" column
hold=[]
for x in customer_df.index:
    if type(customer_df.tel_total_charges[x])== str:
        hold.append(0)
    else:
        hold.append(customer_df.tel_total_charges[x])
customer_df.tel_total_chargs=hold
#Filling NA values for churn reason
customer_df.tel_churn_reason=customer_df.tel_churn_reason.fillna("No reason given")

<ipython-input-31-488fb0b675c0>:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  customer_df.tel_total_chargs=hold


In [32]:
#Binning age 
customer_df.age=pd.cut(x=customer_df['age'], bins=[10,20,30,40, 50,60,70,80], labels=["10","20","30","40", "50","60","70"])

In [33]:
conditions=[
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==0),            
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==1),
]
choises=[
    "Not Churned",
    "No Reason",

]
customer_df.churn_categor=np.select(conditions,choises,default=customer_df.churn_categor)

In [35]:
# Re categorizing the churn reason for better interpretation.
customer_df["tel_churn_reason"]=reason_cleanup("tel_churn_reason",customer_df)

In [36]:
#converting columns into binary
col_to_bin=["senior_citizen","partner","dependents","phone_service","device_protection","tech_support","paperless_billing","referred_a_friend","device_protection_plan","premium_tech_support","streaming_music","unlimited_data","under_30","tel_internet_service","tel_online_securit","tel_online_backup","tel_multiple_lines","tel_streaming_tv","tel_streaming_movies"]
for x in col_to_bin:
    to_binary(x,customer_df)

In [37]:
customer_df.drop(["tel_churn_reason","churn_categor","customer_status"],axis=1, inplace=True)

In [38]:
customer_df=pd.get_dummies(customer_df,columns=["cit","zip_code","latitude","longitude","gender","offer","internet_type","age","tel_payment_method","tel_contract"],drop_first=True)

In [39]:
customer_df=customer_df.astype("float")

In [42]:
#Splitting the data into X and Y
y=customer_df["churn_value"]
X=customer_df[customer_df.columns[customer_df.columns!="churn_value"]]

In [43]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2020, test_size=0.2)

In [44]:
#Choosing Parameter for the model.
params={
    "penalty":["l1","l2"],
    "class_weight":["balanced"],
    "max_iter":range(100,5000,100),
    "warm_start":["True","False"]   
}

In [45]:
# using vanilla Logistic Regression model and keeping the Random state for continuity and reproducibility
log_reg=LogisticRegression(random_state=40)
grid_lg=GridSearchCV(log_reg, params, cv=10, scoring="f1", verbose=1, n_jobs=-1)

In [ ]:
grid_lg.fit(X_train,y_train)

Fitting 10 folds for each of 196 candidates, totalling 1960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 73.9min


In [ ]:
print("Best Gridsearch Score: {}".format(grid_lg.best_score_))
print("Best Parameters: {}".format(grid_lg.best_params_))
print("Best Estimators: {}".format(grid_lg.best_estimator_))

In [ ]:
gs_pred=grid_lg.best_estimator_.predict(X_test)

In [ ]:
gs_acc=metrics.recall_score(y_test, gs_pred)
gs_rec=metrics.accuracy_score(y_test, gs_pred)
gs_f1=metrics.f1_score(y_test,gs_pred)

In [ ]:
print('GridSearch Logistic Regression Accuracy: {}'.format(gs_acc))
print('GridSearch Logistic Regression Recall: {}'.format(gs_rec))
print('GridSearch Logistic Regression F1: {}'.format(gs_f1))

In [ ]:
pred = confusion_matrix(y_test, gs_pred)
plt.figure(figsize=(8,8))

sns.heatmap(pred.T, square=True, annot=True, fmt='d',cmap="PuBu", cbar=False,
            xticklabels=['Stayed', 'Churned'], yticklabels=['Stayed', 'Churned'])
sns.set(font_scale=2)
plt.xlabel('True Values',fontsize=20)
plt.ylabel('Predicted Values',fontsize=20);
plt.savefig('/Users/carlosruiz/Desktop/Mod_5_project/Images/matrix', dpi=300)

In [ ]:
coef=pd.DataFrame(zip(X_train.columns, np.transpose(grid_lg.best_estimator_.coef_[0])), columns=['features', 'coef'])
coef.iloc[list(coef.coef.sort_values(ascending=False).index)].reset_index(drop=True,inplace=True)